# Rainfall and Estimating Statistical Significance 
At the suggestion of a friend, I am completing assignment 5 from COMP 116 from UNC.  

The course website, courtesy of Professor Gary Bishop, can be found at:  
https://wwwx.cs.unc.edu/Courses/comp116-s16/A5.html  

---

According to [this NASA article](https://www.nasa.gov/topics/earth/features/midweek_rainfall.html 'NASA Data Link Pollution to Rainy Summer Days in the Southeast'), there is more rainfall during the week than on weekends from summer storms in the Southeastern US. The goal of this assignment is to see whether rainfall data from our local airport (RDU) supports this.

Professor Bishop provided 10 years of daily rainfall data in a csv file for his class, but it does not appear to be available on the course page, so I will first get that data.

Specific instructions given are as follows:  
1. Read the data.
2. Determine the days of the week from the dates.
3. Write a function to get the average daily rainfall for the midweek (T-R) and the weekend (S-M).<br>
(I'm not sure why Friday is left out, but both the assignment and the NASA article say "midweek" is Tuesday through Thursday, and "weekend" is Saturday through Monday. [Well, the assignment gives them those labels, and the NASA article refers to those day ranges.])
4. Report the average daily rainfall for the midweek and weekend, and the difference between these.
5. Determine and report the p-value by simulation.
---

I downloaded a daily summary for January 1, 2009 through January 2, 2019 for RDU from
[NOAA](https://www.ncdc.noaa.gov/cdo-web/search) (technically placed an order which cost nothing; it was very quick).

In [1]:
# import packages
import numpy as np
import pandas as pd # version 0.23 is required for dt.day_name()
import matplotlib.pyplot as plt

### 1. Read the data.

In [2]:
# load the data
rdu_data = pd.read_csv('rdu_weather_data_2009_2019.csv')

In [3]:
rdu_data.head()

,STATION,NAME,DATE,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN
0,USW00013722,"RALEIGH AIRPORT, NC US",2009-01-01,0.0,0.0,0.0,NaN,5.6,-5.0
1,USW00013722,"RALEIGH AIRPORT, NC US",2009-01-02,1.3,0.0,0.0,NaN,5.6,-2.8
2,USW00013722,"RALEIGH AIRPORT, NC US",2009-01-03,0.0,0.0,0.0,NaN,14.4,0.0
3,USW00013722,"RALEIGH AIRPORT, NC US",2009-01-04,15.7,0.0,0.0,NaN,13.3,6.1
4,USW00013722,"RALEIGH AIRPORT, NC US",2009-01-05,0.0,0.0,0.0,NaN,15.6,11.7


In [4]:
# I selected the station ID (although it's all from RDU), date, precipitation, snowfall,
# snow depth, average temp, max temp, and min temp.
# Average temp seems to be empty until april of 2013

### 2. Determine days of the week.

In [5]:
# the DATE column is an object
rdu_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653 entries, 0 to 3652
Data columns (total 9 columns):
STATION    3653 non-null object
NAME       3653 non-null object
DATE       3653 non-null object
PRCP       3653 non-null float64
SNOW       3653 non-null float64
SNWD       3653 non-null float64
TAVG       2102 non-null float64
TMAX       3653 non-null float64
TMIN       3653 non-null float64
dtypes: float64(6), object(3)
memory usage: 256.9+ KB


In [6]:
# convert the DATE column to a datetime object
rdu_data['DATE'] = pd.to_datetime(rdu_data['DATE'])
rdu_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3653 entries, 0 to 3652
Data columns (total 9 columns):
STATION    3653 non-null object
NAME       3653 non-null object
DATE       3653 non-null datetime64[ns]
PRCP       3653 non-null float64
SNOW       3653 non-null float64
SNWD       3653 non-null float64
TAVG       2102 non-null float64
TMAX       3653 non-null float64
TMIN       3653 non-null float64
dtypes: datetime64[ns](1), float64(6), object(2)
memory usage: 256.9+ KB


In [7]:
# use the DATE column to get the days of the week into a new column
rdu_data['DAY'] = rdu_data['DATE'].dt.day_name()
rdu_data.head()

,STATION,NAME,DATE,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,DAY
0,USW00013722,"RALEIGH AIRPORT, NC US",2009-01-01,0.0,0.0,0.0,NaN,5.6,-5.0,Thursday
1,USW00013722,"RALEIGH AIRPORT, NC US",2009-01-02,1.3,0.0,0.0,NaN,5.6,-2.8,Friday
2,USW00013722,"RALEIGH AIRPORT, NC US",2009-01-03,0.0,0.0,0.0,NaN,14.4,0.0,Saturday
3,USW00013722,"RALEIGH AIRPORT, NC US",2009-01-04,15.7,0.0,0.0,NaN,13.3,6.1,Sunday
4,USW00013722,"RALEIGH AIRPORT, NC US",2009-01-05,0.0,0.0,0.0,NaN,15.6,11.7,Monday


### 3. Write a function to get the average daily rainfall for the midweek (T-R) and the weekend (S-M).

In [8]:
# define these lists out here so I can use them elsewhere, too
midweek = ['Tuesday', 'Wednesday', 'Thursday'] # see you later, Friday?
weekend = ['Saturday', 'Sunday', 'Monday']

# define a function that takes a dataframe and the names of the rainfall and day columns, 
# and returns the midweek and weekend average rainfalls
# along with the number of days used to compute those averages
def rain_avgs(df, prcp, day):
    
    mid_avg = df[df[day].isin(midweek)][prcp].mean()
    end_avg = df[df[day].isin(weekend)][prcp].mean()
    
    return mid_avg, end_avg

### 4. Report the average daily rainfall for the midweek and weekend, and the difference between them.

In [10]:
# Get the midweek and weekend averages for RDU from 2009 to 2019
mid_avg, end_avg = rain_avgs(rdu_data, 'PRCP', 'DAY')

# find the difference
diff_avg = abs(mid_avg - end_avg)

# print the above values
print(f'The average daily rainfall was {mid_avg:.2f} cm from Tue through Thu between 2009 and 2019')
print(f'The average daily rainfall was {end_avg:.2f} cm from Sat through Mon between 2009 and 2019')
print(f'The absolute difference in the average daily rainfall between the midweek and weekend was {diff_avg:.2f} cm')

The average daily rainfall was 3.07 cm from Tue through Thu between 2009 and 2019
The average daily rainfall was 3.62 cm from Sat through Mon between 2009 and 2019
The absolute difference in the average daily rainfall between the midweek and weekend was 0.55 cm


### 5. Determine and report the p-value by simulation.

In [11]:
# test the null hypothesis
# (that there is no difference in the average rainfall between the midweek and weekend)

# by running the above fcn many times with different permutations of the days,
# (i.e., random assigning days to be labeled "midweek" or "weekend", essentially)

# and counting how many times the difference between the means is greater than the difference
# reported above between the midweek and weekend groups

In [12]:
# I made the rain_avgs fcn to specifically compare midweek to weekend days
# and it's not a complicated fcn, so I'm not actually going to use it to do the permutation testing
# because it seems more complicated to try to do that

In [17]:
# number of permutation trials
N = 1000

# number of permutations that produced a greater difference
count = 0

# initiate a list to hold the (randomly simulated) differences
sim_diffs = []

for i in range(N):
    # shuffle the rows in the df
    df = rdu_data.set_index(np.random.permutation(rdu_data.index)).sort_index()

    # get the mean precip of the first half of the rows
    # (split the df into two parts, index into the zeroth part, and take the mean of the PRCP column)
    mean1 = np.array_split(df,2)[0]['PRCP'].mean()

    # get the mean precip of the second half of the rows
    # (same as just above but index into the index 1 split of the df)
    mean2 = np.array_split(df,2)[1]['PRCP'].mean()
    
    # calculate the difference in the mean rainfalls
    mean_diff = abs(mean1 - mean2)
    
    # put that difference into the list of simulated differences
    sim_diffs.append(mean_diff)

# find the number of simulated differences that had a greater difference than the actual difference
count = len([x for x in sim_diffs if x >= diff_avg])

# report findings
print(f'Out of {N} permutation trials, {count} ({(100*count/N):.1f}%) had a difference in rainfall greater')
print('(than or equal to the actual difference between midweek and weekend rainfalls)')
print('')
print(f'That is, the p-value is {(count/N):.3f}')

Out of 1000 permutation trials, 85 (8.5%) had a difference in rainfall greater
(than or equal to the actual difference between midweek and weekend rainfalls)

That is, the p-value is 0.085


### Epilogue: Conclusions
With a p-value above 0.05, we conclude that there is not a statistically significant difference in the rainfall on weekdays (T/W/R) and on weekends (S/S/M).

However, as Professor Bishop noted, NASA did this with all of the southeast, while we used data only from the RDU weather station. So, I am going to pull more data from NOAA and see what I get from that.

---

The tool I was using to get weather data from NOAA is limited to 1,000 station-years, and selecting the southeast region includes nearly 8,000 stations. So, I would need to submit almost 80 separate requests to get all of the data I want.

I am going to look for an alternate source of weather data and then come back to this.